<a href="https://colab.research.google.com/github/chu-ise/378A-2022/blob/main/notebooks/03/01_bok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Environment setup to persist data in Google Drive

In [6]:
import os, sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    if not os.path.exists('/content/drive'):
      drive.mount("/content/drive")
      !ln -s "/content/drive/My Drive/colab_workspace" workspace
    WORKSPACE_DIR = "/content/workspace"
else:
    WORKSPACE_DIR = "../../workspace" 
print(f'WORKSPACE_DIR = {WORKSPACE_DIR}')
data_dir = os.path.join(WORKSPACE_DIR, "data")
os.makedirs(data_dir, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ln: failed to create symbolic link 'workspace/My': File exists
ln: failed to create symbolic link 'workspace/colab_workspace': File exists
WORKSPACE_DIR = /content/workspace


In [1]:
import gdown, os
id = "142Cd7boufftU56nIgbOnqerxWcgQblsO"

data_file = "conf.zip"
conf_dir = data_file.split(".")[0]
if not os.path.exists(conf_dir):
    gdown.download(id=id, output=data_file, quiet=False)
    !unzip $data_file
    os.remove(data_file)
else:
    print(f"{conf_dir} already exists")

conf already exists


In [2]:
%%capture
%pip install ekorpkit[dataset]==0.1.12
%pip install ekorpkit[bok]==0.1.12

### Download Bank of Korea Minutes and Build a Dataset

In [3]:
from ekorpkit.io.fetch.loader.bok import BOKMniutes

In [4]:
import os
from hydra import initialize, compose

In [7]:
with initialize(config_path="conf"):
    cfg=compose(
        overrides=[
            "+downloader=bok",
            f"+data_dir={data_dir}",
        ]
    )
    print(cfg)

{'downloader': {'name': 'bok_minutes', 'output_dir': '${..data_dir}/${.name}', 'output_file': '${.name}.csv', 'autoload': False, 'force_download': True, 'num_workers': 1, 'compress': False, 'verbose': True, '_target_': 'ekorpkit.io.fetch.loader.bok.BOKMniutes', 'scrap': {'base_url': 'http://bok.or.kr', 'url': 'https://www.bok.or.kr/portal/bbs/B0000245/news.rss?menuNo=200761', 'from_date': 20171201, 'file_prefix': 'BOK', 'raw_hwp_dir': '${..output_dir}/raw/hwp', 'raw_txt_dir': '${..output_dir}/raw/txt'}}, 'data_dir': '/content/workspace/data'}


In [8]:
bm = BOKMniutes(**cfg.downloader)

In [9]:
bm.build()

/content/workspace/data/bok_minutes/raw/hwp/BOK_20220224_20220315.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20220114_20220203.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20211223_20220107.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20211209_20211228.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20211125_20211214.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20211012_20211102.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20210924_20211012.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20210909_20210928.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20210826_20210914.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20210729_20210817.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20210715_20210803.hwp already exists
/content/workspace/data/bok_minutes/raw/hwp/BOK_20210630_20210720

In [11]:
BOKMniutes.build??

In [12]:
data_file =  cfg.downloader.output_dir + "/bok_minutes.csv"
print(data_file)


/content/workspace/data/bok_minutes/bok_minutes.csv


In [17]:
import pandas as pd
df = pd.read_csv(data_file)
df.sort_values(by=['mdate'], axis=0, ascending=False, inplace=True)
df.head()

,id,filename,mdate,rdate,section,text
55,BOK_20220224_20220315_S2,BOK_20220224_20220315,2022-02-24 10:00:00,2022-03-15 16:00:00,Foreign Currency,"일부 위원은 신흥국에서 자산가격 하락, 외화유동성 부족 등의 문제가 불거졌던 과거 ..."
56,BOK_20220224_20220315_S3,BOK_20220224_20220315,2022-02-24 10:00:00,2022-03-15 16:00:00,Financial Markets,NaN
57,BOK_20220224_20220315_S4,BOK_20220224_20220315,2022-02-24 10:00:00,2022-03-15 16:00:00,Monetary Policy,일부 위원은 기준금리의 실물경제 파급시차를 고려하면 앞으로 그 영향이 확대될 수 있...
58,BOK_20220224_20220315_S5,BOK_20220224_20220315,2022-02-24 10:00:00,2022-03-15 16:00:00,Participants’ Views,일부 위원은 금번 회의에서 기준금리를 현 수준인 1.25%로 동결하는 것이 적절하다...
59,BOK_20220224_20220315_S6,BOK_20220224_20220315,2022-02-24 10:00:00,2022-03-15 16:00:00,Government’s View,NaN


In [18]:
df.info() # check for missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 204 entries, 55 to 172
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        204 non-null    object
 1   filename  204 non-null    object
 2   mdate     204 non-null    object
 3   rdate     204 non-null    object
 4   section   204 non-null    object
 5   text      136 non-null    object
dtypes: object(6)
memory usage: 11.2+ KB
